In [2]:
# -*- coding: utf-8 -*-
# 埼玉千葉神奈川記事反映本番用
import requests
import json
from wordpress_xmlrpc.methods import media
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import media, posts
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc import Client, WordPressPost
from PIL import Image, ImageDraw, ImageFont
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import os
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
import urllib
from bs4 import BeautifulSoup
import re
import csv
import codecs
import requests
import urllib.request as req
import glob
import json
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import rcParams
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import timedelta
import datetime
import os
from dotenv import load_dotenv
from selenium.webdriver.common.by import By
from wordpress_xmlrpc import Client, WordPressPost, methods
from src.utils import *
print('ライブラリの読み込み完了')

# .envファイルの内容を読み込見込む
load_dotenv()



def get_concat_h_blank(im1, im2, color=(27, 44, 74)):
    dst = Image.new('RGB', (im1.width + im2.width, max(im1.height, im2.height)), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


def get_concat_v(im1, im2, color=(0, 0, 0)):
    dst = Image.new('RGB', (max(im1.width, im2.width), im1.height + im2.height), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


def get_concat_v_blank(im1, im2, color=(0, 0, 0)):
    dst = Image.new('RGB', (max(im1.width, im2.width), im1.height + im2.height), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


def upload_image(in_image_file_name, out_image_file_name):
    if os.path.exists(in_image_file_name):
        with open(in_image_file_name, 'rb') as f:
            binary = f.read()

        data = {
            "name": out_image_file_name,
            "type": 'image/png',
            "overwrite": True,
            "bits": binary
        }

        media_id = wp.call(media.UploadFile(data))['id']
        print(in_image_file_name.split('/')
              [-1], 'Upload Success : id=%s' % media_id)
        return media_id
    else:
        print(in_image_file_name.split('/')[-1], 'NO IMAGE!!')

def post_line(message):
    url = "https://notify-api.line.me/api/notify"
    token = os.environ['LINE_TOKEN']
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    #print('image_path',image_path)
    #files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload) 



読み込み完了
ライブラリの読み込み完了


In [38]:

WORDPRESS_ID = os.environ['WORDPRESS_ID']
WORDPRESS_PW = os.environ['WORDPRESS_PW']
WORDPRESS_URL = os.environ['WORDPRESS_URL']
wp = Client(WORDPRESS_URL, WORDPRESS_ID, WORDPRESS_PW)

In [3]:
twitter = UtilsTwitterClass()

In [4]:
twitter.add_target_date(1)
vars(twitter)

{'tomorrow_8numbers_string': '20230326',
 'tomorrow_date_stinrg_jp': '3/26(日)',
 'tomorrow_date_string_sql': '2023-03-26',
 'yesterday': datetime.date(2023, 3, 24),
 'tomorrow': datetime.date(2023, 3, 26),
 'yesterday_date_string': '3/24(金)',
 'id': '',
 'pw': '',
 'tweet_text': '',
 'image_path_list': [],
 'main_tweet_text': '',
 'project_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site',
 'image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image',
 'tweet_footer_text': '\n\n#XXXX 20230324\n\n',
 'created_image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image\\image',
 'servise_account_json_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\config\\twitteranalytics-jsonsercretkey.json',
 'target_date': datetime.date(2023, 3, 26),
 'target_date_string_jp': '3月26日(日)',
 'target_date_string_sql': '2023-03-26'}

In [39]:
target_date
prefecture ='愛知県'

In [40]:
def generate_thumbnail(target_date:str,prefecture:str) -> str:

    image_path = r'image\image_source\千葉.jpg'  # mac
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    write_image_context = f'''{target_date}
{prefecture}
取材予定まとめ'''

    font_path = r"font/LightNovelPOPv2.otf"
    # sizeは文字サイズです（とりあえず適当に50）
    font = ImageFont.truetype(font_path, size=240)
    draw.multiline_text((130, 20), write_image_context, fill=(255, 255, 255),align='center',font=font, spacing=50, stroke_width=5, stroke_fill=(55, 55, 55))
    save_image_path = fr'image\thumbnail_{target_date}_{prefecture}.jpg'
    image.save(save_image_path)
    return save_image_path


In [32]:
#recommend_image2()

#upload_image(save_image_jpg, save_image_jpg.replace('image/全媒体/',''))



wp = Client(WORDPRESS_URL, WORDPRESS_ID, WORDPRESS_PW)

# for i in range(1,4):
#     imgPath = win_path+f'heatmap/tokyo_heatmap_{tenpo_name}_{date_list[0]}_{i}.jpg'
#     upload_image(imgPath, imgPath)
#     time.sleep(1)


# import
generated_thumbnail_path = generate_thumbnail(target_date,prefecture)
output_thumnail_path = f'thumbnail_{target_date}_{prefecture}.jpg'
# Picture file name & Upload
media_id = upload_image(generated_thumbnail_path, output_thumnail_path)

# Blog Title

title = f"【{prefecture}】{target_date} パチンコスロットイベント取材まとめ"
# Blog Content (html)
body = f'''
テスト
'''

# publish or draft
status = 'draft' # "publish"　or# 'draft'

# Category keyword
cat1 = '取材予定まとめ'
cat2 = ''
cat3 = ''

# Tag keyword
tag1 = f'{prefecture}'
tag2 = f'{target_date}'

slug = f"pledge_report_{target_date}_{prefecture}"

# Post
post = WordPressPost()
post.title = title
post.content = body
post.post_status = status
post.terms_names = {
    "category": [cat1],
    "post_tag": [tag1, tag2],
}
post.slug = slug

# Set eye-catch image

post.thumbnail = media_id

# Post Time
#post.date = datetime.datetime.now() - datetime.timedelta(hours=9)

wp.call(NewPost(post))
print('記事書き込み完了')
post_line(f'{prefecture}_{target_date}分がブログ投稿完了')
# except Exception as e:
#     post_line(f'{tomorrow}分ブログ投稿失敗\n{e}')

image\thumbnail_3月12日(火)_愛知県.jpg Upload Success : id=37412
記事書き込み完了


In [43]:
# 20件ずつ取得したい場合、
number = 20  # 一度に取得するコンテンツ数
offset = 0  # オフセット
while True:
    posts = wp.call(methods.posts.GetPosts({"number": number, "offset": offset}))
    if len(posts) == 0:
        break  # これ以上取得するコンテンツがない場合

    for post in posts:
        print(post.id, post.title)

    offset += number
    if offset > 10:
        break

37413 【3月12日(火)】愛知県 パチンコスロットイベント取材まとめ
37410 【3/29(水)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37269 
37382 【3/28(火)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37355 【3/27(月)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37324 【3/26(日)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37296 【3/25(土)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37267 【3/24(金)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37240 【3/23(木)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37213 【3/22(水)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37186 【3/21(火)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37159 【3/20(月)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37131 【3/19(日)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37104 【3/18(土)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37077 【3/17(金)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37050 【3/16(木)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
37023 【3/15(水)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
36996 【3/14(火)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
36969 【3/13(月)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
36942 【3/12(日)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】


In [ ]:
for 

title = f"【{prefecture}】{target_date} パチンコスロットイベント取材まとめ"